Импортируем всякое

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from scipy import interp
from sklearn.metrics import roc_auc_score

Вдруг придётся преоткрыть

In [2]:
train = None
test = None
def reload_data():
    global train
    train = pd.read_csv("../data/raw/application_train.csv")
    global test
    test = pd.read_csv("../data/raw/application_test.csv")

Избавимся от y\n во флагах

In [3]:
def flag_to_int(df):
    for i in df:
        if "FLAG" in i:
            df[i] = np.where((df[i] == 'Y') | (df[i] == 'y') | (df[i] == 1), 1, 0)

In [4]:
reload_data()

In [5]:
flag_to_int(train)
flag_to_int(test)

In [6]:
target = "TARGET"

Достаём часть данных, чтобы ускорить работу

In [7]:
def eq_df(df):
    positive = df[df[target] == 1]
    negative = df[df[target] == 0]

    indexes1 = np.random.choice(len(positive), replace=False, size=20000)
    indexes2 = np.random.choice(len(negative), replace=False, size=20000)


    newdf = pd.concat([positive.iloc[indexes1], negative.iloc[indexes2]]).join(pd.read_csv("../data/raw/bureau.csv"), on='SK_ID_CURR', how='inner', rsuffix='_b').join(pd.read_csv("../data/raw/credit_card_balance.csv"), on='SK_ID_CURR', how='inner', rsuffix='_ccb').join(pd.read_csv("../data/raw/previous_application.csv"), on='SK_ID_CURR', how='inner', rsuffix='_pa').join(pd.read_csv("../data/raw/POS_CASH_balance.csv"), on='SK_ID_CURR', how='inner', rsuffix='_pcb')
    return newdf
lite = eq_df(train)
flag_to_int(lite)

Функция для тестиования и измерения orc auc

In [19]:
def comp_roc(df, features, target = "TARGET"):
    kf = KFold(n_splits=5, shuffle=True) #random_state=42)
    
    roc_list = []
    for i, (train_index, val_index) in enumerate(kf.split(df)):
        train_part = df.iloc[train_index, : ]
        train_x = train_part[features]
        train_y = train_part[target]
        
        val_part = df.iloc[val_index, :]
        test_x = val_part[features]
        test_y = val_part[target]
        
        model = LogisticRegression()
        
        model.fit(train_x, train_y)
        
        y_score = model.predict_proba(test_x)[:,1]

        fpr, tpr, _ = roc_curve(test_y, y_score)
        roc_auc = auc(fpr, tpr)

        print(f'Fold full {i}: roc_auc {roc_auc}, count values {y_score}')
        roc_list.append(roc_auc)

        
    print(f'roc average = {np.mean(roc_list)}, std = {np.std(roc_list)}')

Функция для тестирования и подсчёта msle

In [10]:

def comp_mse(df, features, target):
    kf = KFold(n_splits=5, shuffle=True) #random_state=42)
    
    msle_list = []
    mse_list = []
    for i, (train_index, val_index) in enumerate(kf.split(df)):    
        train_part = df.iloc[train_index, : ]
        val_part = df.iloc[val_index, :]
        model = SGDRegressor()
        model.fit(X=train_part[features].fillna(0), y = train_part[target])

        val_predication = model.predict(val_part[features].fillna(0)).clip(0, 100000000000)

        mse = mean_squared_error(val_predication, val_part[target])
        msle = mean_squared_log_error(val_predication, val_part[target])

        
        print(f'Fold full {i}: msle {msle}, mse {mse}')
        msle_list.append(msle)
        mse_list.append(mse)
        
    print(f'MSLE average = {np.mean(msle_list)}, std = {np.std(msle_list)}')
    

In [34]:
def predict_n_write_csv(features, path="predict1.csv"):
    target = "TARGET"
    
    model = LogisticRegression()
    model.fit(X=lite[features].fillna(0), y = lite[target])
    
    sample_submission = pd.read_csv("../data/raw/sample_submission.csv")
    sample_submission[target] = model.predict(test[features]) 
    sample_submission.to_csv(path, index= False)

Подготовка данных

In [12]:
def prepare_data(x1, x2=None):
    global train
    train = x1(train)
    if x2 != None:
        global test
        test = x2(test)

Подцепим флаги из осного сета

In [13]:
flags = ["FLAG_MOBIL", "FLAG_EMP_PHONE", "FLAG_CONT_MOBILE", "FLAG_EMAIL", "REG_REGION_NOT_WORK_REGION",
        # "DAYS_LAST_PHONE_CHANGE", 'REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION',
         'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY',
            'FLAG_OWN_REALTY', 'FLAG_OWN_CAR']

In [14]:
lite.AMT_CREDIT_SUM.fillna(0, inplace=True)

In [15]:
lite['have_credit'] = lite.AMT_CREDIT_SUM > lite.AMT_GOODS_PRICE

In [16]:
lite['have_credit'] = np.where(lite['have_credit'], 1, 0)


In [17]:
lite[['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3", target]]

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET
182376,NaN,0.243033,0.000527,1
305296,0.456198,0.705166,0.472253,1
280705,0.029535,0.649452,0.159520,1
112440,0.247591,0.560507,0.164414,1
102235,NaN,0.540059,0.107532,1
...,...,...,...,...
17238,NaN,0.564489,0.683269,0
134991,0.195874,0.484197,NaN,0
1225,0.428096,0.638422,NaN,0
216783,0.572976,0.625233,0.782608,0


Посмотрим как соотносятся EXT_SOURCE и target

In [20]:
comp_roc(lite[['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3", target]].dropna(), ['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3"])

Fold full 0: roc_auc 0.7474018475750577, count values [0.76098931 0.61518053 0.69850231 ... 0.14829457 0.15438935 0.27847492]
Fold full 1: roc_auc 0.7522412819710118, count values [0.73819858 0.57161397 0.64086602 ... 0.25997207 0.39438873 0.17973408]
Fold full 2: roc_auc 0.7367768355766015, count values [0.79210436 0.70829096 0.4861368  ... 0.55703008 0.2958644  0.2814512 ]
Fold full 3: roc_auc 0.7318127687815948, count values [0.34228634 0.70810965 0.63343899 ... 0.52631443 0.33790979 0.49580532]
Fold full 4: roc_auc 0.7374474850464758, count values [0.43917089 0.32036963 0.26478586 ... 0.23664477 0.14210132 0.14002826]
roc average = 0.7411360437901483, std = 0.007511567491068526


In [21]:
predict_n_write_csv(['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3"], 'all_ext_souce.csv')

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Увы есть пропуски, попробуем вычислсить.

In [22]:
comp_mse(lite[flags + ['EXT_SOURCE_1']].dropna(), flags, 'EXT_SOURCE_1')

Fold full 0: msle 0.019718981994631644, mse 0.040749902639176036
Fold full 1: msle 0.020219399959246213, mse 0.04165768693029467
Fold full 2: msle 0.02045182534731501, mse 0.04213181774866574
Fold full 3: msle 0.02032239115909918, mse 0.041839467841941654
Fold full 4: msle 0.020928057457742885, mse 0.042427292244028164
MSLE average = 0.020328131183606987, std = 0.00038939181860837005


### Кажется подбор $\pm$ успешен

In [23]:
def fillna_with_model(df, model, features, target):
    df[target] = np.where(df[target].isna(), model.predict(df[features]), df[target])

In [39]:
def fillna_EXT_SOURCE():
    model = SGDRegressor()
    target = 'EXT_SOURCE_1'
    part = train[flags + [target]].dropna()
    model.fit(X=part[flags], y=part[target])
    fillna_with_model(train, model, flags, target)
    fillna_with_model(test, model, flags, target)
    fillna_with_model(lite, model, flags, target)
    
    target = 'EXT_SOURCE_2'
    model = SGDRegressor()
    part =train[flags + [target]].dropna()
    model.fit(X=part[flags], y=part[target])
    fillna_with_model(train, model, flags, target) 
    fillna_with_model(test, model, flags, target)
    fillna_with_model(lite, model, flags, target)
    
    target = 'EXT_SOURCE_3'
    model = SGDRegressor()
    part = train[flags + [target]].dropna()
    model.fit(X=part[flags], y=part[target])
    fillna_with_model(train, model, flags, target)
    fillna_with_model(test, model, flags, target)
    fillna_with_model(lite, model, flags, target)

In [40]:
fillna_EXT_SOURCE()

In [26]:
comp_roc(train[['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3", target]].dropna(), ['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3"])

Fold full 0: roc_auc 0.7250331172613247, count values [0.10752731 0.09199032 0.03379605 ... 0.06097241 0.05112123 0.07883791]
Fold full 1: roc_auc 0.7249839693423139, count values [0.03737098 0.02868642 0.11451819 ... 0.05543631 0.03790669 0.08179572]
Fold full 2: roc_auc 0.7204496023651097, count values [0.0336585  0.06237655 0.03092111 ... 0.04982343 0.02787348 0.10091856]
Fold full 3: roc_auc 0.7198996316547865, count values [0.07136159 0.0460263  0.06860566 ... 0.03770265 0.13624582 0.05470569]
Fold full 4: roc_auc 0.7195598026331512, count values [0.41980413 0.02690139 0.04088272 ... 0.08011082 0.06092931 0.08846672]
roc average = 0.7219852246513372, std = 0.002484859151066652


## Private
0.49998
## Public
0.50072



In [27]:
predict_n_write_csv(['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3"], 'all_ext_souce.csv')

Попробуем ещё вариант с EXT_SOURC

In [28]:
comp_roc(train[['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3", target]].dropna(), ['EXT_SOURCE_1'])
comp_roc(train[['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3", target]].dropna(), ['EXT_SOURCE_2'])
comp_roc(train[['EXT_SOURCE_1', 'EXT_SOURCE_2',"EXT_SOURCE_3", target]].dropna(), ['EXT_SOURCE_3'])

Fold full 0: roc_auc 0.6072782730009684, count values [0.09764174 0.0770024  0.09265694 ... 0.0770024  0.14303117 0.04931171]
Fold full 1: roc_auc 0.608266019472773, count values [0.19820182 0.08527503 0.12002143 ... 0.03107257 0.09700092 0.04591481]
Fold full 2: roc_auc 0.6017333685962084, count values [0.12272244 0.07896533 0.0764458  ... 0.08520441 0.04882693 0.04447816]
Fold full 3: roc_auc 0.6093856224184209, count values [0.07696415 0.04187491 0.07696415 ... 0.05712044 0.14007487 0.08566633]
Fold full 4: roc_auc 0.6161536911447825, count values [0.08819531 0.0867322  0.04791243 ... 0.08457599 0.07694071 0.17284373]
roc average = 0.6085633949266306, std = 0.004619045029574435
Fold full 0: roc_auc 0.6548513252635919, count values [0.05094348 0.11619631 0.04205438 ... 0.06181256 0.07468551 0.10971935]
Fold full 1: roc_auc 0.6602969243781613, count values [0.15236087 0.03973955 0.06316058 ... 0.05582312 0.07256307 0.05088337]
Fold full 2: roc_auc 0.651587179564524, count values [0.04

In [29]:
def predict_by_one(df, feature, df2):
    model = LogisticRegression()
    
    model.fit(X=df[feature], y=df[target])
    predictation = feature[0] + '_predict'
    df[predictation] = model.predict(df[feature])
    df2[predictation] = model.predict(df2[feature])

In [42]:
predict_by_one(lite, ['EXT_SOURCE_1'],test)
predict_by_one(lite, ['EXT_SOURCE_2'],test)
predict_by_one(lite, ['EXT_SOURCE_3'],test)

In [43]:
comp_roc(lite[['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict", target]].dropna(), ['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict"])

Fold full 0: roc_auc 0.6746966934352976, count values [0.57110059 0.57110059 0.28594141 ... 0.28594141 0.37787457 0.28594141]
Fold full 1: roc_auc 0.6782299766796269, count values [0.57053742 0.67619974 0.76194119 ... 0.28678249 0.38129095 0.49206194]
Fold full 2: roc_auc 0.6811258591287869, count values [0.76094203 0.57357119 0.76094203 ... 0.28883237 0.28883237 0.38762477]
Fold full 3: roc_auc 0.6720503132012527, count values [0.3848693  0.76564088 0.28474915 ... 0.3848693  0.57816968 0.28474915]
Fold full 4: roc_auc 0.6809763550610222, count values [0.67522138 0.57079731 0.38305929 ... 0.59817872 0.76125461 0.38305929]
roc average = 0.6774158395011972, std = 0.003557722975085349


### Private
0.50000

### Public
0.50000

Похоже на ошибку, но я пока не понял

Предсказываются одни нули, так что буду предсказавать на срезе данных:
### Private
0.62106

### Public
0.63881


In [44]:
predict_n_write_csv(['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict"], 'all_ext_souce_plus.csv')

Раз мы можем получить из флагов EXT_SOURCE, значит из них можем получить и финальный результат:

In [46]:
comp_roc(lite[flags + [target]].dropna(), flags)

Fold full 0: roc_auc 0.5709818423818923, count values [0.51563711 0.54768168 0.51563711 ... 0.4005529  0.49074022 0.57988209]
Fold full 1: roc_auc 0.5773416742731232, count values [0.57761343 0.46378924 0.57615728 ... 0.51891925 0.52040836 0.64438059]
Fold full 2: roc_auc 0.5706275964120919, count values [0.52074993 0.58160087 0.40143668 ... 0.51573792 0.45430535 0.40143668]
Fold full 3: roc_auc 0.5629730356107545, count values [0.65120279 0.51374827 0.58105669 ... 0.58105669 0.51374827 0.51161865]
Fold full 4: roc_auc 0.575361979320325, count values [0.56689211 0.48335633 0.51420509 ... 0.40424474 0.51420509 0.51420509]
roc average = 0.5714572255996374, std = 0.0049551910431407985


Ерунда какая-то:
### Private
0.50000

### Public
0.50000

После перехода на срез

### Private
0.55456

### Public
0.55075

такое себе

In [47]:
predict_n_write_csv(flags, 'just_fkags.csv')

Попробую добваить новый флаг

In [48]:
lite["have_big_credit"] = lite['AMT_CREDIT_SUM'] > lite['AMT_GOODS_PRICE']
lite["have_big_credit"] = np.where(lite["have_big_credit"], 1, 0)

In [71]:
comp_roc(lite[['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict"] + [target, 'have_big_credit']].dropna(), ['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict", 'have_big_credit'])

Fold full 0: roc_auc 0.6708606839783684, count values [0.76594103 0.28688151 0.76594103 ... 0.38022312 0.28688151 0.28688151]
Fold full 1: roc_auc 0.6792527236176533, count values [0.38604686 0.57431643 0.67446014 ... 0.38604686 0.38604686 0.57431643]
Fold full 2: roc_auc 0.6770236116569712, count values [0.57129511 0.76345168 0.57438873 ... 0.28473817 0.38510204 0.38510204]
Fold full 3: roc_auc 0.6862841155093589, count values [0.38476884 0.28771757 0.59565936 ... 0.57275963 0.57275963 0.28771757]
Fold full 4: roc_auc 0.6751127676199407, count values [0.67858476 0.5757384  0.49172427 ... 0.28764334 0.28764334 0.49172427]
roc average = 0.6777067804764585, std = 0.00509951519380594


### Private
0.62106

### Public
0.63881

ничего не поменялось

In [52]:
test=test.join(pd.read_csv("../data/raw/bureau.csv"), on='SK_ID_CURR', how='inner', rsuffix='_b')
# train=train.join(pd.read_csv("../data/raw/bureau.csv"), on='SK_ID_CURR', how='inner', rsuffix='_b')


In [54]:
lite["have_big_credit"] = lite['AMT_CREDIT_SUM'] > lite['AMT_GOODS_PRICE']
lite["have_big_credit"] = np.where(lite["have_big_credit"], 1, 0)

test["have_big_credit"] = test['AMT_CREDIT_SUM'] > test['AMT_GOODS_PRICE']
test["have_big_credit"] = np.where(test["have_big_credit"], 1, 0)

In [55]:
flags.append('have_big_credit')

In [72]:
predict_n_write_csv(['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict", 'have_big_credit'], 'just_flags_plus.csv')

In [57]:
def count_docs(df):
    df['DOCUMENT_FLAGS_COUNT'] = df['FLAG_DOCUMENT_2'] + df['FLAG_DOCUMENT_3'] + df['FLAG_DOCUMENT_4'] + df['FLAG_DOCUMENT_5'] + df['FLAG_DOCUMENT_6'] + df['FLAG_DOCUMENT_7'] + df['FLAG_DOCUMENT_8'] + df['FLAG_DOCUMENT_9'] + df['FLAG_DOCUMENT_10'] + df['FLAG_DOCUMENT_11'] + df['FLAG_DOCUMENT_12'] + df['FLAG_DOCUMENT_13'] + df['FLAG_DOCUMENT_14'] + df['FLAG_DOCUMENT_15'] + df['FLAG_DOCUMENT_16'] + df['FLAG_DOCUMENT_17'] + df['FLAG_DOCUMENT_18'] + df['FLAG_DOCUMENT_19'] + df['FLAG_DOCUMENT_20'] + df['FLAG_DOCUMENT_21']

In [61]:
count_docs(train)
count_docs(test)
count_docs(lite)

In [74]:
comp_roc(lite[['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict", "DOCUMENT_FLAGS_COUNT", target]], ['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict", "DOCUMENT_FLAGS_COUNT"])

Fold full 0: roc_auc 0.6834854006556528, count values [0.57231214 0.57231214 0.72809982 ... 0.2545449  0.3436628  0.28925956]
Fold full 1: roc_auc 0.6760622405858367, count values [0.57240697 0.68294327 0.71775335 ... 0.38131817 0.38131817 0.28944933]
Fold full 2: roc_auc 0.6756963892548024, count values [0.76683531 0.28866229 0.76683531 ... 0.76683531 0.60056205 0.33605697]
Fold full 3: roc_auc 0.6839099694742815, count values [0.38430017 0.57510512 0.76221219 ... 0.44158822 0.38430017 0.292343  ]
Fold full 4: roc_auc 0.6748105579028507, count values [0.67725561 0.57687875 0.60784681 ... 0.73122265 0.49610764 0.52891265]
roc average = 0.6787929115746849, std = 0.004027602031414368


### Private
0.50000

### Public
0.50000

### Private
0.55686

### Public
0.55395

In [75]:
predict_n_write_csv(['EXT_SOURCE_1_predict', 'EXT_SOURCE_2_predict',"EXT_SOURCE_3_predict", "DOCUMENT_FLAGS_COUNT"], 'flags_docs.csv')

In [65]:
comp_roc(lite[flags + ["DOCUMENT_FLAGS_COUNT", 'AMT_INCOME_TOTAL', target]].fillna(0), flags + ['DOCUMENT_FLAGS_COUNT', 'AMT_INCOME_TOTAL'])

Fold full 0: roc_auc 0.5218828132813281, count values [0.49996789 0.49995184 0.49997324 ... 0.49996254 0.4999786  0.49996789]
Fold full 1: roc_auc 0.5259735896716794, count values [0.49998716 0.49998288 0.4999786  ... 0.49994008 0.49998288 0.49998545]
Fold full 2: roc_auc 0.5244563208198887, count values [0.49996584 0.4999573  0.49997438 ... 0.49994876 0.49994022 0.4998941 ]
Fold full 3: roc_auc 0.5239340472943759, count values [0.49468473 0.48287909 0.48819005 ... 0.48376401 0.49261795 0.49261795]
Fold full 4: roc_auc 0.522609745719385, count values [0.49990491 0.49986687 0.49988589 ... 0.49994675 0.49992393 0.49986687]
roc average = 0.5237713033573315, std = 0.0014325788241386292


### Private
0.5

### Public
0.5

опять фигня

In [66]:
predict_n_write_csv(flags + ['DOCUMENT_FLAGS_COUNT', 'AMT_INCOME_TOTAL'], 'flags_docs_money.csv')